<a href="https://colab.research.google.com/github/anonymous0905/Capstone-flowchart-summary-module/blob/main/Cleaned_Gemma_2_Fine_tuning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q -U transformers
%pip install -q -U datasets
%pip install -q -U accelerate
%pip install -q -U peft
%pip install -q -U trl
%pip install -q -U bitsandbytes
%pip install -U wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 21.5 MB/s eta 0:00:00
   

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install jsonlines

In [4]:
import itertools
import jsonlines
from datasets import load_dataset
from pprint import pprint
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer
import pandas as pd

In [5]:
filename = "/content/sampled_data.jsonl"
instruction_dataset_df = pd.read_json(filename, lines=True)
examples = instruction_dataset_df.to_dict()


prompt_template_with_input = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a summary of the input that includes all key details such as information about the plaintifs and defendants, key events including dates and other details such as statutes etc.

### Input:
{input}

### Response:"""

num_examples = len(examples["judgment"])
finetuning_dataset = []

#hydrating the dataset using the defined prompt template
for i in range(num_examples):
    judgment = examples["judgment"][i]
    summary = examples["summary"][i]
    text_with_prompt_template = prompt_template_with_input.format(input=judgment)
    finetuning_dataset.append({"input": text_with_prompt_template, "response": summary})

from pprint import pprint
print("One datapoint in the finetuning dataset:")
pprint(finetuning_dataset[0])


One datapoint in the finetuning dataset:
{'input': 'Below is an instruction that describes a task, paired with an input '
          'that provides further context. Write a response that appropriately '
          'completes the request.\n'
          '\n'
          '### Instruction:\n'
          'Generate a summary of the input that includes all key details such '
          'as information about the plaintifs and defendants, key events '
          'including dates and other details such as statutes etc.\n'
          '\n'
          '### Input:\n'
          '63 and 64 of 1957. Petitions under Article 71(1) of the '
          'Constitution of India for clarification of doubts in connection '
          'with the election of the President. R. V. section Mani and I. R. V. '
          'Sastri, for the petitioner in Petition No. 63 of 1957. R. Patnaik, '
          'for the petitioner in Petition No. 64 of 1957. M. C. Setalvad '
          'Attorney General for India, G. N. Joshi, Porus A. Mehta a

In [6]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
# Define model ID and configuration
model_id = "google/gemma-2-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16  # Using bfloat16 for better precision
)


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import torch

In [7]:
!pip install flash-attn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 73.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,  # Apply 4-bit quantization
    attn_implementation="flash_attention_2",  # Enable FlashAttention-2
    torch_dtype=torch.bfloat16,  # Use bfloat16 for mixed precision
    device_map={"": 0},  # Map model to GPU 0
    token=os.environ['HF_TOKEN']
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
text = finetuning_dataset[0]['input']
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

# Check input length
input_length = len(inputs['input_ids'][0])
print(f"Input token length: {input_length}")

# Calculate allowable new tokens
max_possible_new_tokens = 1500
if max_possible_new_tokens <= 0:
    raise ValueError("Input text is too long. Please provide a shorter input.")

outputs = model.generate(**inputs, max_new_tokens=max_possible_new_tokens)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Input token length: 4783
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a summary of the input that includes all key details.

### Input:
63 and 64 of 1957. Petitions under Article 71(1) of the Constitution of India for clarification of doubts in connection with the election of the President. R. V. section Mani and I. R. V. Sastri, for the petitioner in Petition No. 63 of 1957. R. Patnaik, for the petitioner in Petition No. 64 of 1957. M. C. Setalvad Attorney General for India, G. N. Joshi, Porus A. Mehta and R. H. Dhebar, for the respondents (Caveators) in both the petitions. May 3. The Judgment of the Court was delivered by DAS C. J. The petitioners in the above petitions have moved this Court to exercise the jurisdiction and power vested in it by and under article 71(1) of the Constitution of India and to inquire into and decide what has been desc

In [ ]:
os.environ["WANDB_DISABLED"] = "false"

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [ ]:
from datasets import load_dataset

# Load the dataset
data = load_dataset("shashankvh24/capstone-summary")

# Tokenize the dataset
data = data.map(lambda samples: tokenizer(samples["judgment"]), batched=True)

# Reduce the training set size to 500 samples
data["train"] = data["train"].select(range(100))

# Verify the new dataset size
print(data)


train_data.csv:   0%|          | 0.00/210M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7030 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'judgment', 'summary', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})


In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'judgment', 'summary', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [ ]:
def formatting_func(example):
    #text = f"Judgement: {example['judgment'][0]}\nSummary: {example['summary'][0]}"
    text = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a summary of the input that includes all key details such as information about the plaintifs and defendants, key events including dates and other details such as statutes etc.

### Input:
{example['judgment'][0]}

### Response:
{example['summary'][0]}
"""
    return [text]

In [ ]:
import torch
from transformers import TrainerCallback

# Custom callback to clear GPU cache after every step
class ClearCacheCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        torch.cuda.empty_cache()

# Modify the trainer to include the callback
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        warmup_steps=1,
        max_steps=5,
        learning_rate=2e-4,
        bf16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="adamw_bnb_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
    callbacks=[ClearCacheCallback()]  # Add the ClearCacheCallback here
)


max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `flash_attention_2`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,2.194400
2,1.925300
3,1.972400
4,2.490700
5,1.889700


TrainOutput(global_step=5, training_loss=2.0944839000701903, metrics={'train_runtime': 35.7375, 'train_samples_per_second': 0.14, 'train_steps_per_second': 0.14, 'total_flos': 140969818856448.0, 'train_loss': 2.0944839000701903, 'epoch': 0.05})

In [ ]:
text = finetuning_dataset[0]['input']
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

# Check input length
input_length = len(inputs['input_ids'][0])
print(f"Input token length: {input_length}")

# Calculate allowable new tokens
max_possible_new_tokens = 1500
if max_possible_new_tokens <= 0:
    raise ValueError("Input text is too long. Please provide a shorter input.")

outputs = model.generate(**inputs, max_new_tokens=max_possible_new_tokens)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Input token length: 4783
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a summary of the input that includes all key details.

### Input:
63 and 64 of 1957. Petitions under Article 71(1) of the Constitution of India for clarification of doubts in connection with the election of the President. R. V. section Mani and I. R. V. Sastri, for the petitioner in Petition No. 63 of 1957. R. Patnaik, for the petitioner in Petition No. 64 of 1957. M. C. Setalvad Attorney General for India, G. N. Joshi, Porus A. Mehta and R. H. Dhebar, for the respondents (Caveators) in both the petitions. May 3. The Judgment of the Court was delivered by DAS C. J. The petitioners in the above petitions have moved this Court to exercise the jurisdiction and power vested in it by and under article 71(1) of the Constitution of India and to inquire into and decide what has been desc

In [ ]:
#saving the model
trainer.save_model("/content/gemma2-2b-it-fine-tuned")


In [ ]:
tokenizer.save_pretrained("/content/gemma2-2b-it-tokenizer")


('/content/gemma2-2b-it-tokenizer/tokenizer_config.json',
 '/content/gemma2-2b-it-tokenizer/special_tokens_map.json',
 '/content/gemma2-2b-it-tokenizer/tokenizer.model',
 '/content/gemma2-2b-it-tokenizer/added_tokens.json',
 '/content/gemma2-2b-it-tokenizer/tokenizer.json')

In [ ]:
!zip -r /content/gemma2-2b-it-finetuned-tokenizer.zip /content/

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 23%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2024.09.19/ (stored 0%)
  adding: content/.config/logs/2024.09.19/13.24.45.437093.log (deflated 58%)
  adding: content/.config/logs/2024.09.19/13.24.57.186451.log (deflated 58%)
  adding: content/.config/logs/2024.09.19/13.24.2

In [10]:
from zipfile import ZipFile
zip_path = '/content/gemma2-2b-it-fine-tuned.zip'
extract_path = '/content/gemma2-2b-it-fine-tuned'
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [11]:
from zipfile import ZipFile
zip_path = '/content/gemma2-2b-it-tokenizer.zip'
extract_path = '/content/gemma2-2b-it-tokenizer'
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('/content/gemma2-2b-it-fine-tuned/gemma2-2b-it-fine-tuned')
model = AutoModelForCausalLM.from_pretrained('/content/gemma2-2b-it-fine-tuned/gemma2-2b-it-fine-tuned')


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [17]:
model.to("cuda:0")

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2304, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2304, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=2304, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): 

In [18]:
text = finetuning_dataset[0]['input']
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

# Check input length
input_length = len(inputs['input_ids'][0])
print(f"Input token length: {input_length}")

# Calculate allowable new tokens
max_possible_new_tokens = 1500
if max_possible_new_tokens <= 0:
    raise ValueError("Input text is too long. Please provide a shorter input.")

outputs = model.generate(**inputs, max_new_tokens=max_possible_new_tokens)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Input token length: 4804


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a summary of the input that includes all key details such as information about the plaintifs and defendants, key events including dates and other details such as statutes etc.

### Input:
63 and 64 of 1957. Petitions under Article 71(1) of the Constitution of India for clarification of doubts in connection with the election of the President. R. V. section Mani and I. R. V. Sastri, for the petitioner in Petition No. 63 of 1957. R. Patnaik, for the petitioner in Petition No. 64 of 1957. M. C. Setalvad Attorney General for India, G. N. Joshi, Porus A. Mehta and R. H. Dhebar, for the respondents (Caveators) in both the petitions. May 3. The Judgment of the Court was delivered by DAS C. J. The petitioners in the above petitions have moved this Court to exercise the jurisdiction and power vested in it by and u